In [ ]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 22.0 MB/s eta 0:00:00
173.7 MiB [] 0% 11.5s173.7 MiB [] 0% 26.5s173.7 MiB [] 0% 23.6s173.7 MiB [] 0% 17.0s173.7 MiB [] 0% 11.2s173.7 MiB [] 0% 9.5s173.7 MiB [] 1% 8.6s173.7 MiB [] 1% 6.4s173.7 MiB [] 2% 4.9s173.7 MiB [] 3% 4.0s173.7 MiB [] 4% 3.5s173.7 MiB [] 5% 3.3s173.7 MiB [] 5% 3.5s173.7 MiB [] 6% 3.2s173.7 MiB [] 7% 3.1s173.7 MiB [] 7% 2.9s173.7 MiB [] 8% 2.9s173.7 MiB [] 9% 2.7s173.7 MiB [] 10% 2.7s173.7 MiB [] 10% 2.6s173.7 MiB [] 11% 2.5s173.7 MiB [] 12% 2.5s173.7 MiB [] 13% 2.4s173.7 MiB [] 15% 2.2s173.7 MiB [] 16% 2.1s173.7 MiB [] 17% 2.0s173.7 MiB [] 18% 2.0s173.7 MiB [] 19% 1.9s173.7 MiB [] 20% 1.8s173.7 MiB [] 21% 1.8s173.7 MiB [] 21% 1.9s173.7 MiB [] 23% 1.8s173.7 MiB [] 24% 1.7s173.7 MiB [] 26% 1.6s173.7 MiB [] 27% 1.5s173.7 MiB [] 29% 1.4s173.7 MiB [] 30% 1.4s173.7 MiB [] 32% 1.3s173.7 MiB [] 33% 1.3s173.7 MiB [] 34% 1.2s173.7 MiB [] 36% 1.2s173.7 MiB [] 37% 1.2s173.7 MiB [] 39% 1.1s173.7 MiB [] 40% 1.1s173.7 MiB 

In [ ]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
import json
from datetime import datetime

async def scrape():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # List to store all product data for DataFrame
        all_products_data = []

        await page.goto("https://jalalsons.com.pk", timeout=60000)
        print("Navigated to site.")

        # --- FIRST ITERATION: Handle popup and select first branch ---
        try:
            if await page.locator("#website_custom_popup").is_visible():
                await page.locator('#website_custom_popup .modal-header a.cursor-pointer.ms-auto').click()
                print("Popup closed.")
        except:
            print("No popup found or already closed.")

        # Click delivery tab and get all branches
        await page.click("a#delivery-loc-tab")
        await page.wait_for_selector("#selectDeliveryBranch", timeout=10000)
        branches = await page.locator("#selectDeliveryBranch option").all_text_contents()
        valid_branches = [b for b in branches if "Please select" not in b and "Lahore" in b]

        print(f"\nFound {len(valid_branches)} valid branches in Lahore")

        # --- Extract category structure once ---
        target_categories = ["BAKERY", "DELI", "JS ICECREAM", "SWEETS", "DEALS", "GROCERY"]

        # Process each branch
        for branch_index, branch_name in enumerate(valid_branches):
            print(f"\n{'='*80}")
            print(f"Processing Branch {branch_index }/{len(valid_branches)}: {branch_name}")
            print(f"{'='*80}")
            print(branch_index)


            if branch_index == 7:
                # First branch: already on the selection page
                print("sucessfully entered")
                await page.select_option("#selectDeliveryBranch", label=branch_name)
                await page.wait_for_timeout(2000)
                await page.click("a#delivery_order")
                print(f"Selected first branch: {branch_name}")
            else:
                # Subsequent branches: click location header to reopen modal
                await page.click("a#get_current_loc")
                await page.wait_for_selector("#selectDeliveryBranch", timeout=10000)
                await page.wait_for_timeout(500)

                # Select the branch
                await page.select_option("#selectDeliveryBranch", label=branch_name)
                await page.wait_for_timeout(2000)
                await page.click("a#delivery_order")
                print(f"Selected branch: {branch_name}")

            await page.wait_for_selector("ul.navbar-nav", timeout=10000)
            nav_items = await page.locator("ul.navbar-nav > li.nav-item").element_handles()

            # Extract category links for this branch
            category_links = {}
            for li in nav_items[:-3]:
                main_cat = await li.query_selector("a.nav-link")
                main_name = (await main_cat.inner_text()).strip()
                if main_name not in target_categories:
                    continue

                await main_cat.hover()
                await page.wait_for_timeout(500)
                sub_links = await li.query_selector_all("ul.dropdown-content a")

                urls = []
                if sub_links:
                    for sub in sub_links:
                        sub_name = (await sub.inner_text()).strip()
                        sub_href = await sub.get_attribute("href")
                        if sub_href:
                            urls.append({
                                "name": sub_name,
                                "url": f"https://jalalsons.com.pk{sub_href}"
                            })
                    category_links[main_name] = urls
                else:
                    href = await main_cat.get_attribute("href")
                    if href:
                        category_links[main_name] = [{
                            "name": main_name,
                            "url": f"https://jalalsons.com.pk{href}"
                        }]

            for main_name, subcats in category_links.items():
                print(f"\nCategory: {main_name}")

                for sub in subcats:
                    sub_name = sub["name"]
                    sub_url = sub["url"]
                    print(f"   Scraping {sub_name}: {sub_url}")

                    try:
                        await page.goto(sub_url, timeout=60000)
                        await page.wait_for_selector(".single_product_theme", timeout=10000)
                    except Exception as e:
                        print(f"   Could not load {sub_name}: {e}")
                        continue

                    products = await page.query_selector_all(".single_product_theme")

                    for product in products:
                        name_el = await product.query_selector("p.product_name_theme")
                        price_el = await product.query_selector("span.price-value")
                        img_el = await product.query_selector("img")

                        name = (await name_el.inner_text()).strip() if name_el else None
                        price = (await price_el.inner_text()).strip() if price_el else None
                        image = await img_el.get_attribute("src") if img_el else None

                        if name and price:
                            # Determine category and subcategory
                            if main_name == sub_name:
                                # No subcategory, main category only
                                category = main_name
                                subcategory = None
                            else:
                                # Has subcategory
                                category = main_name
                                subcategory = sub_name

                            all_products_data.append({
                                "branch": branch_name,
                                "category": category,
                                "subcategory": subcategory,
                                "product_name": name,
                                "price": price,
                                "image_url": image,
                                "product_url": sub_url
                            })

                    print(f"   Scraped {len(products)} products from {sub_name}")

        # --- Create DataFrame ---
        df = pd.DataFrame(all_products_data)

        # Reorder columns for better readability
        column_order = ["branch", "category", "subcategory", "product_name", "price", "image_url", "product_url"]
        df = df[column_order]

        # Save to CSV
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filename = f"jalalsons_products_{timestamp}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

        # Save to Excel for better formatting
        excel_filename = f"jalalsons_products_{timestamp}.xlsx"
        df.to_excel(excel_filename, index=False, sheet_name='Products')

        # Print summary statistics
        print(f"\n{'='*80}")
        print("SCRAPING SUMMARY")
        print(f"{'='*80}")
        print(f"Total products scraped: {len(df)}")
        print(f"Total branches: {df['branch'].nunique()}")
        print(f"Total categories: {df['category'].nunique()}")
        print(f"\nProducts per branch:")
        print(df['branch'].value_counts())
        print(f"\nProducts per category:")
        print(df['category'].value_counts())
        print(f"\nData saved to:")
        print(f"   - {csv_filename}")
        print(f"   - {excel_filename}")

        # Display first few rows
        print(f"\nSample data:")
        print(df.head(10).to_string())

        await browser.close()

# Run
await scrape()